Formation OpenClassrooms DS-IML - **Marc Lefèvre**, <marc.lefevre@noos.fr>

# **Projet 7 : Classification de textes avec BERT**

## **2ème partie : Construction d'un plus grand dataset**

Suite aux résultats obtenus lors de notre première série de modélisations (notebook précédent), nous allons reconstruire ici un nouveau dataset plus large que l'original. Nous estimons que procéder à de nouvelles modélisations à partir de plus de données pourrait en effet nous permettre d'obtenir de meilleurs résultats.

### **Importation des bibliothèques python**

In [1]:
import pandas as pd
import os
import pickle
import bs4
import spacy
import re
import numpy as np
import random

from time import time
from bs4 import BeautifulSoup as soupe
from collections import Counter
from spacy.matcher import PhraseMatcher
from sklearn.feature_extraction.text import CountVectorizer

### **Récupération des données**

Téléchargement des données brutes depuis le site **Stack Exchange** et son **API**:<br>https://data.stackexchange.com/stackoverflow/query/new


Cela s'est fait au moyen des requêtes SQL qui suivent, qui nous permettaient de télécharger des données au format **CSV**. Comme précédemment, nous avons uniquement pioché dans les question postérieures à l'année **2014**. Afin de récupérer plus de données que lors du **projet 5**, nous avons diminué le nombre de vues exigées des questions (de 40000 à 20000).

Première requête :

Requêtes suivantes (qui parcourent la base de données Stack Overflow par "paquets" de 5 millions d'entrées...).

Fichiers obtenus :

In [4]:
cwd = os.getcwd()
path = cwd + "/Original_Data_1/"
files = os.listdir(path)
files

['QueryResults(1).csv',
 'QueryResults(2).csv',
 'QueryResults(3).csv',
 'QueryResults(4).csv',
 'QueryResults(5).csv',
 'QueryResults(6).csv',
 'QueryResults(7).csv',
 'QueryResults(8).csv',
 'QueryResults.csv']

Nombre de questions téléchargées :

In [5]:
nb_posts = 0

for f in files :
    
    fich = path + f
    df = pd.read_csv(fich)
    nb_posts = nb_posts + len(df)
    
print(f"Le nombre total de questions téléchargées est : {nb_posts}")

Le nombre total de question téléchargées est : 60387


Réunion des questions dans un dataframe unique.

In [95]:
df = pd.DataFrame()

cwd = os.getcwd()
path = cwd + "/Original_Data_1/"
files = os.listdir(path)

for f in files :
    
    fichier = path + f
    df = pd.concat([df, pd.read_csv(fichier)], ignore_index = True)
    
print(f"Le nombre total de questions téléchargées est : {len(df)}")

Le nombre totale de question téléchargées est : 60387


On a le même nombre de question, la réunion s'est donc bien passée.

In [96]:
df.head(1)

,Id,CreationDate,ViewCount,Score,AnswerCount,CommentCount,FavoriteCount,Tags,Title,Body
0,30000088,2015-05-02 08:28:27,30813,12,3,4,0.0,<java>,How to implement negative indexes in java?,"<p>In Python, you are allowed to use negative ..."


Comme nous aimons les "comptes ronds", nous allons restreindre nos données aux **50000 questions**, parmi celles dont on dispose, qui ont le plus grand nombre de vues.<br>Par ailleurs, avec ce nombre questions, nous aurons doublé celui dont on disposait initialement.

In [100]:
# Classement des questions par nombre de vues
df = df.sort_values("ViewCount", ascending = False)

# On ne garde que les 50000 premières
df = df.iloc[:50000]
df.head(1)

,Id,CreationDate,ViewCount,Score,AnswerCount,CommentCount,FavoriteCount,Tags,Title,Body
60303,29973357,2015-04-30 16:43:35,2893381,990,29,3,626.0,<code-formatting><visual-studio-code>,How do you format code in Visual Studio Code (...,<p>What is the equivalent of <kbd>Ctrl</kbd> +...


Restriction du dataset aux données qui nous intéressent, les colonnes **Tags**, **Title** et **Body**.

In [102]:
df = df[["Tags", "Title", "Body"]]
df.head()

,Tags,Title,Body
60303,<code-formatting><visual-studio-code>,How do you format code in Visual Studio Code (...,<p>What is the equivalent of <kbd>Ctrl</kbd> +...
15907,<git><credentials><git-config><git-extensions>,How to save username and password in Git?,<p>I want to use a push and pull automatically...
20762,<git><rebase>,Git refusing to merge unrelated histories on r...,<p>During <code>git rebase origin/development<...
53759,<python><pip><python-wheel><downloadfile><jpype>,How do I install a Python package with a .whl ...,<p>I'm having trouble installing a Python pack...
15054,<javascript><ajax><http><cors><http-status-cod...,Response to preflight request doesn't pass acc...,<p>I'm getting this error using ngResource to ...


In [103]:
# Sauvegarde des données brutes
pickle_out = open("Data/corpus_raw.pickle", "wb")
pickle.dump(df, pickle_out)
pickle_out.close()

### **Nettoyage des données texte**

In [77]:
df = pickle.load(open("Data/corpus_raw.pickle", "rb"))

Fonction de "nettoyage" des données texte de la colonne **Body**.<br>Cette fonction :<br>- Elimine les "gros" blocs de code présents entre balises de préformattage "pre".<br>- Nettoye le texte gardé des retours charriot.<br><br>Initialement lors du projet 5, nous enlevions aussi les majuscules, mais comme les modèles de type Bert sont sensés traiter du texte tel quel, nous nous passons de cette étape.

In [79]:
def map_text(x):
    
    liste_sans_pre = []
    texte = ""
    
    soup = soupe(x)

    for cont in soup :

        if cont.name != "pre":

            liste_sans_pre.append(cont)
            
    for c in liste_sans_pre :
    
        if isinstance(c, bs4.element.Tag):

            texte = texte + c.text.replace("\n", "") + " "

    texte = texte.rstrip()
    
    #return texte.lower()
    return texte

In [80]:
df["Body_texte"] = df["Body"].map(map_text)
df.head(3)

,Tags,Title,Body,Body_texte
60303,<code-formatting><visual-studio-code>,How do you format code in Visual Studio Code (...,<p>What is the equivalent of <kbd>Ctrl</kbd> +...,What is the equivalent of Ctrl + K + F and Ctr...
15907,<git><credentials><git-config><git-extensions>,How to save username and password in Git?,<p>I want to use a push and pull automatically...,I want to use a push and pull automatically in...
20762,<git><rebase>,Git refusing to merge unrelated histories on r...,<p>During <code>git rebase origin/development<...,During git rebase origin/development the follo...


Ajout d'une ponctuation pour les titres n'en ayant pas.

In [82]:
for index, row in df.iterrows():
    if row['Title'][-1] not in [".", "?", "!"]:
        row['Title'] = row['Title'] + "."

Réunion des colonnes **Title** et **Body** en une seule contenant tout le texte des questions.

In [84]:
df["texte"] = df["Title"] + " " + df["Body_texte"]

In [85]:
df.head(3)

,Tags,Title,Body,Body_texte,texte
60303,<code-formatting><visual-studio-code>,How do you format code in Visual Studio Code (...,<p>What is the equivalent of <kbd>Ctrl</kbd> +...,What is the equivalent of Ctrl + K + F and Ctr...,How do you format code in Visual Studio Code (...
15907,<git><credentials><git-config><git-extensions>,How to save username and password in Git?,<p>I want to use a push and pull automatically...,I want to use a push and pull automatically in...,How to save username and password in Git? I wa...
20762,<git><rebase>,Git refusing to merge unrelated histories on r...,<p>During <code>git rebase origin/development<...,During git rebase origin/development the follo...,Git refusing to merge unrelated histories on r...


In [83]:
i = 0

for index, row in df.iterrows():
    if row['Title'][-1] not in [".", "?", "!"]:
        i = i+1

print(i)

0


Le traitement des tags sera lui inchangé...

Premier nettoyage des tags (chevrons...) et élimination des colonnes désormais inutiles.

In [86]:
def map_tags(x):
    
    x = x.replace("<", "")
    x = x.replace(">", " ")
    x = x.rstrip()
    
    return x

In [87]:
df["tags"] = df["Tags"].map(map_tags)

In [88]:
df = df[["tags", "texte"]]
df.head(5)

,tags,texte
60303,code-formatting visual-studio-code,How do you format code in Visual Studio Code (...
15907,git credentials git-config git-extensions,How to save username and password in Git? I wa...
20762,git rebase,Git refusing to merge unrelated histories on r...
53759,python pip python-wheel downloadfile jpype,How do I install a Python package with a .whl ...
15054,javascript ajax http cors http-status-code-405,Response to preflight request doesn't pass acc...


In [89]:
pickle_out = open("Data/corpus.pickle", "wb")
pickle.dump(df, pickle_out)
pickle_out.close()

### **Nettoyage et sélection des tags**

In [90]:
df = pickle.load(open("Data/corpus.pickle", "rb"))

Chargement d'un modèle **SpaCy** et fonction de nettoyage & uniformisation des tags.

In [91]:
nlp = spacy.blank("en")

def clean_tags(x) :
    
    # supprime les "-"
    x = x.replace("-", " ")
    # regex pour ne garder que les lettres + exceptions : le "#" de "c#"
    x = re.sub('[^a-z#+\s.]+', '', x)
    # on clean les "x" seuls
    x = x.replace(" x", "")

    
    out = ""
    seen = set()
    doc = nlp(x)
    
    for word in doc :
        
        if word.text not in seen:
            out = out + " " + word.text
            out = out.lstrip()
            
        seen.add(word.text)
    
    # corrections manuelles de découpages faits par spacy 
    out = out.replace(" .x", "")
    out = out.replace(" . ", " ")
    out = out.replace("c #", "c#")
    
    return out

Utilistion de cette fonction pour créer une colonne de tags "nettoyés".

In [92]:
df["tags_c"] = df["tags"].map(clean_tags)

Afin de sélectionner les tags les plus courants, on commence par les réunir dans un (long) objet **string** à partir duquel **SpaCy** nous permettra de facilement les compter.

In [93]:
tag_c_text = ""

for t in df.tags_c.values :
    
    tag_c_text = tag_c_text + " " + t
    
tag_c_text = tag_c_text.lstrip()

Décompte des tags...

In [94]:
# Ce code enlève le caractère "#" des suffixes de SpaCy afin de préserver un tag comme "c#"
suffixes = list(nlp.Defaults.suffixes)
suffixes.remove("#")
suffix_regex = spacy.util.compile_suffix_regex(suffixes)
nlp.tokenizer.suffix_search = suffix_regex.search

# Comme on travaille sur un très long objet string, on augmente une limitation par défaut de SpaCy.
nlp.max_length = 1500000

doc2 = nlp(tag_c_text)
tags_c = [token.text for token in doc2 if token.is_punct == False and token.is_space == False]

Hit-parade de nos tags...

In [95]:
tag_c_freq = Counter(tags_c)
top_c = tag_c_freq.most_common(26)
top_c

[('python', 6895),
 ('javascript', 6283),
 ('java', 4438),
 ('android', 4418),
 ('angular', 2985),
 ('html', 2364),
 ('php', 2242),
 ('c#', 2159),
 ('reactjs', 2116),
 ('studio', 2009),
 ('css', 1821),
 ('ios', 1675),
 ('node.js', 1649),
 ('spring', 1521),
 ('typescript', 1511),
 ('swift', 1372),
 ('jquery', 1358),
 ('asp.net', 1300),
 ('laravel', 1263),
 ('sql', 1237),
 ('react', 1223),
 ('visual', 1200),
 ('google', 1143),
 ('json', 1135),
 ('pandas', 1125),
 ('docker', 936)]

Sans les chiffres...

In [96]:
top = [t[0] for t in top_c]
top, len(top)

(['python',
  'javascript',
  'java',
  'android',
  'angular',
  'html',
  'php',
  'c#',
  'reactjs',
  'studio',
  'css',
  'ios',
  'node.js',
  'spring',
  'typescript',
  'swift',
  'jquery',
  'asp.net',
  'laravel',
  'sql',
  'react',
  'visual',
  'google',
  'json',
  'pandas',
  'docker'],
 26)

On enlève le terme "studio" qui est un terme parasite...

In [97]:
liste = ["studio"]

for m in liste :
    
    top.remove(m)

Le traitement plus tard du doublon "reactjs / react" diminuera notre nombre de tags à 24. Mais l'ajout d'un tag "misc" pour toutes les questions n'ayant pas de tag dans cette liste nous fera revenir à 25.

Déterminons maintenant grâce à un **PhraseMatcher** de **SpaCy** si les questions ont ou pas un de leurs tags dans notre "top".

In [99]:
df["tag_in_top"] = 0

matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp(tag) for tag in top]
matcher.add("TOP", patterns) 

for idx, quest in df.iterrows() :
    
    doc = nlp(quest.tags_c)
    matches = matcher(doc)
    if len(matches) != 0 :
        df["tag_in_top"][idx] = 1

<ipython-input-99-ac5a801dd9ba>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tag_in_top"][idx] = 1


In [100]:
df.tail(5)

,tags,texte,tags_c,tag_in_top
6316,mysql ubuntu,unable to connect to mysql database in Ubuntu....,mysql ubuntu,0
16711,javascript html css,Add Multiple Styles with JavaScript. I want to...,javascript html css,1
6571,mips,What is the use of a $zero register in MIPS? W...,mips,0
58042,c# asp.net asp.net-mvc,Deploying an ASP.NET MVC project to server. I'...,c# asp.net mvc,1
6679,swift uigesturerecognizer,How to add a double tap Gesture Recognizer in ...,swift uigesturerecognizer,1


In [101]:
print(f"Nombre des questions sans aucun tag dans le top : {len(df[df.tag_in_top == 0])}")

Nombre des questions sans aucun tag dans le top : 11365


In [102]:
res = (len(df[df.tag_in_top == 0])/ len(df))*100 
print(f"Proportion de questions sans aucun tag dans le top : {res:.2f} %")

Proportion de questions sans aucun tag dans le top : 22.73 %


Création d'une nouvelle colonne qui contiendra pour chaque questions leurs tags appartenant à notre "top" ou un nouveau tag "misc" si elles n'en ont aucun.

In [103]:
df["new_t"] = ""

matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp(tag) for tag in top]
matcher.add("TOP", patterns) 

for idx, quest in df.iterrows() :
    
    doc = nlp(quest.tags_c)
    matches = matcher(doc)
    
    if len(matches) == 0 :
        df["new_t"][idx] = "misc"
        
    else :
                
        tag_str = ""
        for m in matches :    
    
            m_id, m_start, m_end = m
            tag_str = tag_str + " " + doc[m_start:m_end].text
            tag_str = tag_str.lstrip()
            
        df["new_t"][idx] = tag_str

<ipython-input-103-1ea853c472a7>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["new_t"][idx] = tag_str
<ipython-input-103-1ea853c472a7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["new_t"][idx] = "misc"


In [104]:
df.head(3)

,tags,texte,tags_c,tag_in_top,new_t
60303,code-formatting visual-studio-code,How do you format code in Visual Studio Code (...,code formatting visual studio,1,visual
15907,git credentials git-config git-extensions,How to save username and password in Git? I wa...,git credentials config extensions,0,misc
20762,git rebase,Git refusing to merge unrelated histories on r...,git rebase,0,misc


On ne sépare des colonnes qui ne nous intéressent plus.

In [105]:
df = df[["texte", "new_t"]]
df = df.rename(columns={"new_t" : "tags"})

Ne nous reste plus qu'à gérer le doulon "reactjs/react" avec la fonction suivante :

In [106]:
def map_react(x):
    
    x = x.replace("reactjs", "react")
    
    out = ""
    seen = set()
    doc = nlp(x)
    
    # gestion doublon - cas où du coup on se retrouve avec 2x react en tag
    for word in doc :
        
        if word.text not in seen:
            out = out + " " + word.text
            out = out.lstrip()
            
        seen.add(word.text)
    
    return out

In [107]:
df.tags = df.tags.map(map_react)

In [108]:
df.head(20)

,texte,tags
60303,How do you format code in Visual Studio Code (...,visual
15907,How to save username and password in Git? I wa...,misc
20762,Git refusing to merge unrelated histories on r...,misc
53759,How do I install a Python package with a .whl ...,python
15054,Response to preflight request doesn't pass acc...,javascript
24829,How do I kill the process currently using a po...,misc
8811,"TypeError: a bytes-like object is required, no...",python
570,How do I get into a Docker container's shell? ...,docker
43320,"Could not find module ""@angular-devkit/build-a...",angular
29203,MySQL Error: : 'Access denied for user 'root'@...,sql


In [109]:
# Sauvegarde du fichier
pickle_out = open("Data/data.pickle", "wb")
pickle.dump(df, pickle_out)
pickle_out.close()

### **Echantillonnage features/labels**

In [110]:
df = pickle.load(open("Data/data.pickle", "rb"))

In [111]:
liste_data = []

for idx, row in df.iterrows():
    
    liste_data.append((row.texte, row.tags))

In [112]:
random.seed(47)
random.shuffle(liste_data)
random.shuffle(liste_data)

In [113]:
feat = [t[0] for t in liste_data]
targ = [t[1] for t in liste_data]

In [114]:
len(feat), len(targ)

(50000, 50000)

In [115]:
# Sauvegardes
pickle_out = open("Data/feat.pickle", "wb")
pickle.dump(feat, pickle_out)
pickle_out.close()

pickle_out = open("Data/targ.pickle", "wb")
pickle.dump(targ, pickle_out)
pickle_out.close()

### **Echantillonnage train/test**

In [55]:
feat = pickle.load(open("Data/feat.pickle", "rb"))
targ = pickle.load(open("Data/targ.pickle", "rb"))

In [116]:
feat_train = feat[:40000]
feat_test = feat[40000:]

targ_train = targ[:40000]
targ_test = targ[40000:]

In [117]:
len(feat_train), len(feat_test)

(40000, 10000)

### **Préparation finale des données**

**Vectorisation des labels**

In [118]:
# préparation des cibles
def neutral_tokenizer(tokens):
    
    return tokens.split(" ")

vect_targ = CountVectorizer(tokenizer = neutral_tokenizer).fit(targ_train)

big_y_train = vect_targ.fit_transform(targ_train)
big_y_test = vect_targ.transform(targ_test)

e:\anaconda3\envs\oc2\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [120]:
df_targ = pd.DataFrame(big_y_train.toarray(), columns = vect_targ.get_feature_names())
labels = df_targ.columns.tolist()

In [127]:
# sauvegarde
pickle_out = open("Data/big_y_train.pickle", "wb")
pickle.dump(big_y_train, pickle_out)
pickle_out.close()
# sauvegarde
pickle_out = open("Data/big_y_test.pickle", "wb")
pickle.dump(big_y_test, pickle_out)
pickle_out.close()
# sauvegarde
pickle_out = open("Data/big_df_targ.pickle", "wb")
pickle.dump(df_targ, pickle_out)
pickle_out.close()
# sauvegarde
pickle_out = open("Data/big_labels.pickle", "wb")
pickle.dump(labels, pickle_out)
pickle_out.close()

**Preprocessing des données texte**

Jusqu'à présent, dans tout ce qu'on a fait en **NLP**, une grande importance a été donnée au **preprocessing de données texte** en vue d'en envoyer aux modèles des versions "épurées" de celui-ci. Ce n'est pas le cas avec les modèles de type **BERT** (voir notebook précédent...), nous nous passons donc ce ce travail sur la forme du texte.

In [142]:
# Sauvegardes
pickle_out = open("Data/big_feat_train.pickle", "wb")
pickle.dump(feat_train, pickle_out)
pickle_out.close()

pickle_out = open("Data/big_feat_test.pickle", "wb")
pickle.dump(feat_test, pickle_out)
pickle_out.close()

Nous pouvons maintenant refaire nos modélisations (notebook suivant...) avec deux fois plus de données, et voir si cela nous permettra d'obtenir de meilleurs résultats...

**Fin de la seconde partie.**